In [1]:
#library
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.applications as ap

#choose the GPU
GPU_number = "4"
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = GPU_number  

#directs
img224_dir = './dataset/img224.npy'
img512_dir = './dataset/img512.npy'
label_dir = './dataset/label.npy'

#load label and img
img224 = np.load(img224_dir)
img512 = np.load(img512_dir)
label = np.load(label_dir)

2021-12-14 17:37:00.154894: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
#seperate dataset into train and test
#X224_train, X224_test, y224_train, y224_test = train_test_split(img224,label,test_size=0.2)
#X512_train, X512_test, y512_train, y512_test = train_test_split(img512,label,test_size=0.2)
X224_train = img224
y224_train = label
X512_train = img512
y512_train = label

In [3]:
#to make the label one-hot key encoding
y224_train = tf.keras.utils.to_categorical(y224_train, 4)
#y224_test = tf.keras.utils.to_categorical(y224_test, 4)

y512_train = tf.keras.utils.to_categorical(y512_train, 4)
#y512_test = tf.keras.utils.to_categorical(y512_test, 4)

In [4]:
#checkpoint
checkpoint_filepath = './checkpoint/vgg_scratch'
model_checkpoint_callack = keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_filepath,
    save_weight_only = True,
    save_freq = 'epoch',
    mode = 'auto',
    save_best_only = True,
    monitor = 'val_accuracy'
)

In [5]:
#scratch VGG16
def model_vgg16(X_train, y_train,epochs = 8):
    model = ap.VGG16(weights=None,input_shape=(512,512,3),classes=4)

    callbacks = [keras.callbacks.TensorBoard(log_dir='./logs/vgg_scratch'),model_checkpoint_callack]
    
    model.compile(optimizer='AdaDelta', loss='categorical_crossentropy',metrics=['accuracy'])
    history = model.fit(X_train,y_train,batch_size=20, epochs=epochs, callbacks = callbacks, validation_split = 0.2)

    return model,history

In [ ]:
model_vgg16_m1,history= model_vgg16(X512_train,y512_train,16)

In [ ]:
#model_vgg16_m1.evaluate(X512_test,y512_test,batch_size=20)

In [ ]:
model_vgg16_m1.save_weights('./weights/vgg_scratch_weights.h5')

In [ ]:
#check the accuracy curve
print(history.params)
print(history.history.keys())

{'verbose': 1, 'epochs': 32, 'steps': 120}
dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])


In [ ]:
history.history['val_accuracy']

[0.5566666722297668,
 0.5950000286102295,
 0.70333331823349,
 0.721666693687439,
 0.7433333396911621,
 0.7599999904632568,
 0.7416666746139526,
 0.7916666865348816,
 0.778333306312561,
 0.7516666650772095,
 0.7866666913032532,
 0.8583333492279053,
 0.846666693687439,
 0.8550000190734863,
 0.8333333134651184,
 0.8166666626930237,
 0.8550000190734863,
 0.8733333349227905,
 0.875,
 0.8966666460037231,
 0.9100000262260437,
 0.8816666603088379,
 0.9066666960716248,
 0.8866666555404663,
 0.9133333563804626,
 0.9083333611488342,
 0.9066666960716248,
 0.9016666412353516,
 0.9150000214576721,
 0.9133333563804626,
 0.9116666913032532,
 0.9166666865348816]

In [ ]:
#save the curve
import pandas as pd
his = pd.DataFrame(history.history)
his.to_pickle('./curve/history_vgg_scratch.pkl')